In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Ivar\\Downloads\\text summarzre\\text-summarizer\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\Ivar\\Downloads\\text summarzre\\text-summarizer'

In [12]:
#entity
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataTransforamtionConfig:
     root_dir:Path
     data_path: Path
     tokenizer_name: Path


In [13]:
from textsummarzer.constants import CONFIG_FILE_PATH,CONFIG_FILE_PATH
from textsummarzer.utils.common import read_yaml,create_directory

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=CONFIG_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        
        create_directory([self.config.artifacts_root]) 
    def get_data_transformation_config(self)->DataTransforamtionConfig:
      
      config=self.config.data_transformation
      create_directory([config.root_dir])
      
      data_transformation_config=DataTransforamtionConfig(
        root_dir=config.root_dir ,
         data_path= config.data_path,
         tokenizer_name=config.tokenizer_name,
      )
      
      return data_transformation_config

In [16]:
import os 
from textsummarzer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [29]:
class DataTransforamtion:
    def __init__(self,config:DataTransforamtionConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)
    def convert_examples_to_features(self,example_batch):
        input_encodings =self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    def convert(self):
        data_sasum=load_from_disk(self.config.data_path)
        dataset_samsum_pt=data_sasum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [30]:
#create the pipline
try:
    config=ConfigurationManager()
    DataTransformationConfig=config.get_data_transformation_config()
    data_validation=DataTransforamtion(config=DataTransformationConfig)
    data_validation.convert()

except Exception as e:
    raise e

[2024-10-18 21:03:46,625:INFO:common:yaml file: C:\Users\Ivar\Downloads\text summarzre\text-summarizer\config\config.yaml loaded successfully]
[2024-10-18 21:03:46,630:INFO:common:yaml file: C:\Users\Ivar\Downloads\text summarzre\text-summarizer\config\config.yaml loaded successfully]
[2024-10-18 21:03:46,632:INFO:common:directory: artifacts created]
[2024-10-18 21:03:46,633:INFO:common:directory: artifacts/data_transformation created]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 13229.76 examples/s]
